# imbalanced-learnで不均衡データのサンプリングを行う

## 1. データが不均衡な場合

クラス分類、例えば0：負例と1：正例の二値分類を行う際に、データが不均衡である場合がたびたびあります。

例えば、クレジットカードの取引データで、一つの取引に対して<br>
  不正利用かどうか（不正利用なら1、それ以外は0）<br>
といった値が付与されているカラムがあるとします。

通常、不正利用というのは稀に起こる事象なので、不正利用かどうかが格納されているカラムに関しては<br>
ほとんどが0で、1がほとんどない、という状況になりがちです。

上記の状況で不正利用を予測するようなモデル構築をする場合、目的変数として<br>
  不正利用かどうか<br>
を用いることになりますが、0と1の比率が50%から極度に乖離します（1の比率が0.X%とかになる）。

### 予測モデルの問題

こういったデータで予測モデルを構築すると、往々にして負例だけを予測する（予測値がすべて0になる）モデル<br>
になりがちです。

というのは、不均衡なデータの場合はそれでも<b>「正解率（Accuracy）」が高くなってしまうから</b>です。

例えば、目的変数の内訳が、0が99990件、1が10件の場合に、<br>
すべて0と出力するモデルができたとすると、<br>
正解率は99990 / (99990 + 10) = 99.99%<br>
となります。

このモデルは正解率は高いのですが、<br>
すべての不正利用を見逃す（偽陰性：本当は不正利用（=1）だけれども<br>
不正利用でない（=0）と誤って予測する）ことになり、<br>
不正利用を検知したいという目的には全くそぐわないモデルになっています。

## 2. サンプリングの対策

不正利用を予測したい、つまり誤検出が多少増えてもから不正利用を検出したいという状況では、<br>
サンプリングによって正例と負例の割合を変える、<br>
といった方法が採られます。

つまり、学習に使われる正例の割合を増やすことで偽陰性を減らし、<br>
多少の偽陽性（本当は不正利用していない（=0）けれども不正利用（=1）と誤って予測する）は<br>
出しつつも不正利用も検出できるようにします。

割合を変化させるにあたって、大きく以下の3パターンがあります。
<b>
<ol>
  <li>Under Sampling：負例を減らす</li>
  <li>Over Sampling：正例を増やす</li>
  <li>上記の両方を行う</li>
</ol>
</b>
    
これら割合の変化は、Pythonではimbalanced-learnというライブラリを用いると簡単に行えます。

今回は、このimbalanced-learnを用いてUnder/Over Samplingをどう行うかを簡単に紹介します。

## 3. ライブラリのインストール

インストール方法は以下の二つ。

1) pipが利用できる場合は、簡単インストールを行う。
<pre>
$ pip install -U imbalanced-learn 
</pre>

2) 開発版を使用したい場合は、githubからインストール。
<pre>
$ git clone https://github.com/scikit-learn-contrib/imbalanced-learn.git
$ cd imbalanced-learn
$ python setup.py install
</pre>

## 4. サンプルデータ

不均衡データを人工的に生成します。<br>
こういった人工データは、sklearn.datasets.make_classificationを用いると<br>
簡単に作成できます。

今回、10万件のデータで、正例が10件のデータを以下のようにして作成しました。

In [1]:
from sklearn.datasets import make_classification

df = make_classification(
    n_samples = 100000, # sample size
    n_features = 10,          # 特徴量の数
    n_informative = 2,       # 目的変数のラベルと相関が強い特徴量(Informative fearture）の数
    n_redundant = 0,         # Informative featureの線形結合から作られる特徴量(Redundant fearture）の数
    n_repeated = 0,           # Infomative、Redundant featureのコピーからなる特徴量の数(Repeated feature)
    n_classes = 2,                           # 分類するクラス数
    n_clusters_per_class = 2,       # 1クラスあたりのクラスタ数
    weights = [0.9999, 0.0001],  #  クラスの比率
    flip_y = 0,
    class_sep = 1.0,
    hypercube = True,
    shift = 0.0, 
    scale = 1.0,
    shuffle = True,
    random_state = 71)

weights = [0.9999, 0.0001] <br>
-> クラスの比率。例えば、[0.9, 0.1] と与えると0が90%、1が10%になる

flip_y = 0 <br>
-> クラスのフリップ率。例えば0.01とすると各クラスの1%の符号がランダムに変更される

class_sep = 1.0, hypercube = True<br>
-> 生成アルゴリズムに関係するパラメータ

In [2]:
import numpy as np
import pandas as pd
from pandas import DataFrame, Series

df_raw = DataFrame(df[0], columns = ['var1', 'var2', 'var3', 'var4', 'var5', 'var6', 'var7', 'var8', 'var9', 'var10'])
df_raw['Class'] = df[1]

クラスの割合は、以下のようになっています。

In [3]:
df_raw['Class'].value_counts()

# 出力
#0    99990
#1       10
#Name: Class, dtype: int64

0    99990
1       10
Name: Class, dtype: int64

## 5. ロジスティック回帰で分析してみる

不均衡データをサンプリングしないまま、分類のためのロジスティック回帰モデルを作成してみます。

In [4]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix

# 学習用と検証用に分割
X = df_raw.iloc[:, 0:10]
y = df_raw['Class']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 71)

# モデル構築
mod = LogisticRegression()
mod.fit(X_train, y_train)

# 予測値算出
y_pred = mod.predict(X_test)

正解率（Accuracy）は、以下になります。

In [5]:
print('Accuracy(test) : %.5f' %accuracy_score(y_test, y_pred))

# 出力
#Accuracy(test) : 0.99990

Accuracy(test) : 0.99990


このように、正解率99.99%という、一見精度が良さそうなモデルができています。

しかし、混同行列を出力してみると

In [6]:
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
(tn, fp, fn, tp)

# 出力
#(29997, 0, 3, 0)

(29997, 0, 3, 0)

出力結果から、<br>
　TN（正でない（=0）ものを正でない（=0）と予測する）と<br>
　FN（本当は正（=1）だが正でない（=0）と誤って予測する）<br>
のみに値があり、<br>
　　FP（本当は正でない（=0）ものを正である（=1）と誤って予測する）と<br>
　　TP（正である（=1）ものを正である（=1）と予測する）<br>
が0となっています。

つまり、単にすべて0と予測するモデルになっています。

Precision（正と予測したデータのうち，実際に正であるものの割合：TP / （TP + FP））と<br>
Recall（実際に正であるもののうち，正であると予測されたものの割合：TP / （TP + FN））<br>

を評価してみます。

In [7]:
print('precision : %.4f'%(tp / (tp + fp)))

# 出力
#precision : nan
#recall : 0.0000

precision : nan


/root/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in long_scalars
  """Entry point for launching an IPython kernel.


In [8]:
print('recall : %.4f'%(tp / (tp + fn)))

#recall : 0.0000

recall : 0.0000


計算が不能になっているか、0になっているという、ひどい結果です。

実質意味のある予測ができるモデルではありません。

## 6. Under Sampling

サンプル数　　　負：99990　正：10　のサンプル<br>
ここでは、<b>負例を減らして</b>結果がどう変わるかを見てみます。

imbalanced-learnで提供されているRandomUnderSamplerで、<br>
負例サンプルをランダムに減らし、正例サンプルの割合を10%まで上げます。

In [9]:
# ライブラリ
from imblearn.under_sampling import RandomUnderSampler

# 正例の数を保存
positive_count_train = y_train.sum()
# print('positive count:{}'.format(positive_count_train))とすると7件

# 正例が10％になるまで負例をダウンサンプリング
rus = RandomUnderSampler(ratio={0:positive_count_train*9, 1:positive_count_train}, random_state=71)

# 学習用データに反映
X_train_resampled, y_train_resampled = rus.fit_sample(X_train, y_train)

あとはプロトタイプモデル作成の際と同様、ロジスティック回帰モデルを構築し、性能を見てみます。

In [10]:
# モデル作成
mod = LogisticRegression()
mod.fit(X_train_resampled, y_train_resampled)

# 予測値算出
y_pred = mod.predict(X_test)

# Accuracyと混同行列
print('Confusion matrix(test):\n{}'.format(confusion_matrix(y_test, y_pred)))
print('Accuracy(test) : %.5f' %accuracy_score(y_test, y_pred))

# 出力
#Accuracy(test) : 0.96907
#Confusion matrix(test):
#[[29070   927]
# [    1     2]]

Confusion matrix(test):
[[29070   927]
 [    1     2]]
Accuracy(test) : 0.96907


In [11]:
# PrecisionとRecall
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
print('precision : %.4f'%(tp / (tp + fp)))
print('recall : %.4f'%(tp / (tp + fn)))

# 出力
#precision : 0.0022
#recall : 0.6667

precision : 0.0022
recall : 0.6667


正解率は落ちたものの、PrecisionとRecallが0でない値になりました。

混同行列を見ても、TPが0でなくなっており、FNが小さくなっていることがわかります。<br>
しかし、その代償としてFPが927件と大きくなってしまい、それが小さいPrecisionとして跳ね返っています。

## 7. Over Sampling

サンプル数　　　負：99990　正：10　のサンプル<br>
今度は逆に<b>正例を水増し</b>して正例サンプルの割合を10%まで上げます。

imbalanced-learnで提供されているRandomOverSamplerで行います。

In [12]:
# ライブラリ
from imblearn.over_sampling import RandomOverSampler

# 正例を10％まであげる
ros = RandomOverSampler(ratio = {0:X_train.shape[0], 1:X_train.shape[0]//9}, random_state = 71)

# 学習用データに反映
X_train_resampled, y_train_resampled = ros.fit_sample(X_train, y_train)

/root/anaconda3/lib/python3.6/site-packages/imbalanced_learn-0.4.0.dev0-py3.6.egg/imblearn/utils/validation.py:230: UserWarning: After over-sampling, the number of samples (70000) in class 0 will be larger than the number of samples in the majority class (class #0 -> 69993)
  n_samples_majority))


Under Samplingの場合と同様、モデルを作成して性能を見てみます。

In [13]:
# モデル作成
mod = LogisticRegression()
mod.fit(X_train_resampled, y_train_resampled)

# 予測値算出
y_pred = mod.predict(X_test)

# Accuracyと混同行列
print('Confusion matrix(test):\n{}'.format(confusion_matrix(y_test, y_pred)))
print('Accuracy(test) : %.5f' %accuracy_score(y_test, y_pred))

# 出力
#Accuracy(test) : 0.98983
#Confusion matrix(test):
#[[29693   304]
# [    1     2]]

Confusion matrix(test):
[[29693   304]
 [    1     2]]
Accuracy(test) : 0.98983


In [14]:
# PrecisionとRecall
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
print('precision : %.4f'%(tp / (tp + fp)))
print('recall : %.4f'%(tp / (tp + fn)))

# 出力
#precision : 0.0065
#recall : 0.6667

precision : 0.0065
recall : 0.6667


Under Samplingの場合と比較して、FPの数が若干抑えられており（304件）、<br>
Precisionが若干良くなっています。

## SMOTE

上記のOver Samplingでは、正例を単に水増ししていたのですが、<br>
負例を減らし、正例を増やす、といった考えもあります。

こういった方法の一つに、<b>SMOTE（Synthetic Minority Over-sampling Technique）</b><br>
というアルゴリズムがあります。<br>
imbalanced-learnでは、このSMOTEも提供されているので、ここでも試してみます。

In [15]:
# ライブラリ
from imblearn.over_sampling import SMOTE

# SMOTE
smote = SMOTE(ratio={0:X_train.shape[0], 1:X_train.shape[0]//9}, random_state=71)
X_train_resampled, y_train_resampled = smote.fit_sample(X_train, y_train)

# モデル作成
mod = LogisticRegression()
mod.fit(X_train_resampled, y_train_resampled)

# 予測値算出
y_pred = mod.predict(X_test)

/root/anaconda3/lib/python3.6/site-packages/imbalanced_learn-0.4.0.dev0-py3.6.egg/imblearn/utils/validation.py:230: UserWarning: After over-sampling, the number of samples (70000) in class 0 will be larger than the number of samples in the majority class (class #0 -> 69993)
  n_samples_majority))


In [16]:
# Accuracyと混同行列
print('Confusion matrix(test):\n{}'.format(confusion_matrix(y_test, y_pred)))
print('Accuracy(test) : %.5f' %accuracy_score(y_test, y_pred))

# 出力
#Accuracy(test) : 0.98923
#Confusion matrix(test):
#[[29675   322]
# [    1     2]]

Confusion matrix(test):
[[29675   322]
 [    1     2]]
Accuracy(test) : 0.98923


In [17]:
# PrecisionとRecall
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
print('precision : %.4f'%(tp / (tp + fp)))
print('recall : %.4f'%(tp / (tp + fn)))

# 出力
#precision : 0.0062
#recall : 0.6667

precision : 0.0062
recall : 0.6667


Under/Over Samplingを両方合わせ技でやっているので、<br>
Under SamplingとOver Samplingの間くらいの性能になりました。

## まとめ

・今回はOver Samplingが一番有効でありましたが、データが与えられたときに<br>
　有力な手法はそのデータの性質に依存する部分も大きい。<br>

・なのでどういったサンプリングがよいかは、都度色々試してみて決める必要あり。

imbalanced-learnには、上記3つ以外にもサンプリングの方法が実装されています。<br>

imbalanced-learn API — imbalanced-learn 0.3.0 documentation<br>
http://contrib.scikit-learn.org/imbalanced-learn/stable/api.html

### 参考文献
不均衡データに対するClassification<br>
https://qiita.com/ryouta0506/items/619d9ac0d80f8c0aed92

imbalanced-learnで不均衡なデータのunder-sampling/over-samplingを行う<br>
http://ohke.hateblo.jp/entry/2017/08/18/230000

不均衡データをdownsampling + baggingで補正すると汎化性能も確保できて良さそう<br>
http://tjo.hatenablog.com/entry/2017/08/11/162057

不均衡なデータの分類問題について with Python<br>
http://kamonohashiperry.com/archives/469
